In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from pathlib import Path
from cartopy import crs as ccrs
from goesutils import goes_2_roi, radiance_to_BT, cartopy_pyresample_toggle_extent, trasform_cartopy_extent
from goesutils import norm_im
import metpy
from scipy import stats

In [81]:
day = 103
SS = 'G17'
band = 8
npPath = Path('/tmp/npOut/all_npy2')
ncPath = Path('/sharedData/scratch/')
mc_rows = 1001
mc_cols = 401
#mc_rows = 500
#mc_cols = 200

In [6]:
#DATA_ACM_FILES = Path('../')

#Projections
pc = ccrs.PlateCarree()
mc = ccrs.Mercator()

#ROI
extent_pc = [-109.59326, -102.40674, 8.94659, -8.94656]

# Set up mc extents
target_extent_mc_cartopy = trasform_cartopy_extent(extent_pc, pc, mc)
target_extent_mc_pyresample = cartopy_pyresample_toggle_extent(target_extent_mc_cartopy)

In [79]:
def make_roi_from_nc (inputpath,outputpath, 
                      sat,
                      day,
                      band,
                      rad_prefix='OR_ABI-L1b-RadF-M6C',
                      target_extent_mc_pyresample,
                      overwrite=False):
    if band:
        filetemplate = f'{rad_prefix}{band:02}_G{sat:02}_s2019{day}*.nc'
        datakey='Rad'
    else:
        filetemplate = f'{rad_prefix}_G{sat:02}_s2019{day}*.nc'
        datakey='BCM'
    allfiles = list(inputpath.glob(filetemplate))
    for infilename in allfiles:
        infilename_no_nc = str(infilename).split('.')[0]
        outfile = Path(outputpath)/Path(infilename_no_nc).parts[-1]
        if overwrite or (not Path(str(outfile)+'.npy').exists()):

            dataset = xr.open_dataset(infilename)
            roi_rads = goes_2_roi(dataset,
                           target_extent_mc_pyresample,
                           mc_rows,
                           mc_cols,
                           mc,
                           datakey)
            np.save(outfile,roi_rads)
            dataset.close()
            print(f'Writing file: {str(outfile)+".npy"}')
        else:
            print('File exists')


In [49]:
#make_roi_from_nc('17','103','08')
allfiles = list(ncPath.glob(rad_prefix + str(band).zfill(2) + '_' + SS + '_s2019'+str(day).zfill(3)+'*.nc'))
infilename = allfiles[0]
outfile = Path(str(infilename).split('.')[0])
print(outfile)

/sharedData/scratch/OR_ABI-L1b-RadF-M6C08_G17_s20191030000339_e20191030009405_c20191030009441


In [6]:
rad_prefix + str(band).zfill(2) + '_' + SS + '_s2019'+str(day).zfill(3)

'OR_ABI-L1b-RadF-M6C08_G17_s2019103'

'OR_ABI-L1b-RadF-M6C08_G17_s20191030000339_e20191030009405_c20191030009441'

In [3]:
sat,day,band,rad_prefix=17,103,8,'OR_ABI-L1b-RadF-M6C'
filetemplate = f'{rad_prefix}{band:02}_G{sat:02}_s2019{day}*.nc'
print(filetemplate)

OR_ABI-L1b-RadF-M6C08_G17_s2019103*.nc


In [77]:
%%time
make_roi_from_nc(ncPath,npPath,17,103,'',rad_prefix='OR_ABI-L1b-RadF-M6C')

/sharedData/scratch


IndexError: list index out of range

In [32]:
Path('/tmp/npOut/all_npy2')/Path('/sharedData/scratch/OR_ABI-L1b-RadF-M6C08_G17_s20191030000339_e20191030009405_c20191030009441').parts[-1]

PosixPath('/tmp/npOut/all_npy2/OR_ABI-L1b-RadF-M6C08_G17_s20191030000339_e20191030009405_c20191030009441')

In [80]:
%%time
make_roi_from_nc(Path(ncPath)/Path('ACMFiles'),npPath,17,103,'',rad_prefix='OR_ABI-L2-ACMF-M6')

File exists
Writing file: /tmp/npOut/all_npy2/OR_ABI-L2-ACMF-M6_G17_s20191030010339_e20191030019405_c20191030020000.npy
Writing file: /tmp/npOut/all_npy2/OR_ABI-L2-ACMF-M6_G17_s20191030020339_e20191030029406_c20191030029588.npy
Writing file: /tmp/npOut/all_npy2/OR_ABI-L2-ACMF-M6_G17_s20191030030339_e20191030039405_c20191030039593.npy
Writing file: /tmp/npOut/all_npy2/OR_ABI-L2-ACMF-M6_G17_s20191030040339_e20191030049406_c20191030049595.npy
Writing file: /tmp/npOut/all_npy2/OR_ABI-L2-ACMF-M6_G17_s20191030050339_e20191030059406_c20191030059591.npy
Writing file: /tmp/npOut/all_npy2/OR_ABI-L2-ACMF-M6_G17_s20191030100339_e20191030109406_c20191030109595.npy


KeyboardInterrupt: 

In [71]:
Path(ncPath)/Path('ACMFiles')

PosixPath('/sharedData/scratch/ACMFiles')